# Data Combination

In this exercise we received 3 files about the sales of chocolate in 2020 from the imaginary company Sprint & Lüngli. The company sells 3 types of chocolate: normal, fancy and frozen. Before doing some machine learning with it (next lesson) we want to combine the files in one dataframe. However, the files are in different formats and contain different parts of the data. The exercise is the following: Take the 3 files chocolate1.csv, chocolate2.xlsx and chocolate3.xlsx and create one new dataframe that contains the combination of information of all the three excel files.

Hint1: chocolate1.csv has the time in the format "%d/%m/%Y" and datetime.strptime can convert it to a python datetime object.

Hint2: datetime.datetime(year, month, day) might help for chocolate2.xlsx

Hint3: We are only interested in the sales in 2020, other data can be ignored

In [1]:
import pandas as pd
import datetime

## chocolate1

In [2]:
df = pd.read_csv("chocolate1.csv")
df.head()

,date,chocolate_normal,chocolate_fancy
0,01/01/2020,213,167
1,02/01/2020,330,202
2,03/01/2020,737,360
3,04/01/2020,896,302
4,05/01/2020,552,342


In [3]:
df.dtypes

date                object
chocolate_normal     int64
chocolate_fancy      int64
dtype: object

In [4]:
# lets try to pick one single element of the date column and just convert that. 
d = df.iloc[0].date
datetime.datetime.strptime(d, "%d/%m/%Y")

datetime.datetime(2020, 1, 1, 0, 0)

In [5]:
# create a function with this conversion, which makes it easy to use and better for testing
def convert_timestring_to_datetime(string):
    return datetime.datetime.strptime(string, "%d/%m/%Y")

In [6]:
# apply it to the dataframe
df["datetime"] = df["date"].apply(convert_timestring_to_datetime)
df

,date,chocolate_normal,chocolate_fancy,datetime
0,01/01/2020,213,167,2020-01-01
1,02/01/2020,330,202,2020-01-02
2,03/01/2020,737,360,2020-01-03
3,04/01/2020,896,302,2020-01-04
4,05/01/2020,552,342,2020-01-05
...,...,...,...,...
361,27/12/2020,574,441,2020-12-27
362,28/12/2020,560,320,2020-12-28
363,29/12/2020,484,415,2020-12-29
364,30/12/2020,870,300,2020-12-30


In [ ]:
# Alternative solution:
df["datetime2"] = pd.to_datetime(df["date"], format="%d/%m/%Y")

## chocolate2

In [7]:
df2 = pd.read_excel("chocolate2.xlsx")
df2.head()

,year,month,day,weekday,chocolate_frozen
0,2019,12,30,2,120
1,2019,12,31,3,70
2,2020,1,1,4,61
3,2020,1,2,5,65
4,2020,1,3,6,174


In [8]:
def convert_year_month_day_to_datetime(row):
    return datetime.datetime(row["year"], row["month"], row["day"])

In [9]:
df2["datetime"] = df2.apply(convert_year_month_day_to_datetime, axis=1)
df2.head()

,year,month,day,weekday,chocolate_frozen,datetime
0,2019,12,30,2,120,2019-12-30
1,2019,12,31,3,70,2019-12-31
2,2020,1,1,4,61,2020-01-01
3,2020,1,2,5,65,2020-01-02
4,2020,1,3,6,174,2020-01-03


let's combine the data

In [10]:
# using the merge here will get rid of the dates from 2019, as by default it uses an inner join
df_combo = pd.merge(df, df2, on="datetime")

In [11]:
df_combo

,date,chocolate_normal,chocolate_fancy,datetime,year,month,day,weekday,chocolate_frozen
0,01/01/2020,213,167,2020-01-01,2020,1,1,4,61
1,02/01/2020,330,202,2020-01-02,2020,1,2,5,65
2,03/01/2020,737,360,2020-01-03,2020,1,3,6,174
3,04/01/2020,896,302,2020-01-04,2020,1,4,7,152
4,05/01/2020,552,342,2020-01-05,2020,1,5,1,145
...,...,...,...,...,...,...,...,...,...
361,27/12/2020,574,441,2020-12-27,2020,12,27,1,365
362,28/12/2020,560,320,2020-12-28,2020,12,28,2,194
363,29/12/2020,484,415,2020-12-29,2020,12,29,3,211
364,30/12/2020,870,300,2020-12-30,2020,12,30,4,185


## chocolate 3

In [12]:
df3 = pd.read_excel("chocolate3.xlsx")
df3.head()

,date,daytime,sales normal choc
0,2020-01-01,morning,16.0
1,2020-01-01,afternoon,23.0
2,2020-01-01,night,2.0
3,2020-01-02,morning,NaN
4,2020-01-02,afternoon,87.0


Warning: Some of the values here are NaN. We don't know exactly why that is so our best bet is to ask whoever gave us the data. It might make sense to do some data inputation here and for example use a forward fill, or the value of the last same daytime. Alternatively, since all of those NaN values appear in times where there were few sales, it's possible there wasn't any chocolate sold in that timeframe, so we can assume a 0. For the groupby and sum function we can leave the NaN and they will be treated as a 0.

In [13]:
df3_grouped = df3.groupby(['date']).sum()
df3_grouped

,sales normal choc
date,
2020-01-01,41.0
2020-01-02,98.0
2020-01-03,233.0
2020-01-04,109.0
2020-01-05,170.0
...,...
2020-12-27,170.0
2020-12-28,125.0
2020-12-29,77.0


Merge it together with the other data

In [14]:
# Different options on how to merge these together. This uses the index for 
# the df3_grouped and the datetime column for the df_combo dataframe
df_combo2 = pd.merge(df_combo, df3_grouped, left_on="datetime", right_index=True)
df_combo2.head()

,date,chocolate_normal,chocolate_fancy,datetime,year,month,day,weekday,chocolate_frozen,sales normal choc
0,01/01/2020,213,167,2020-01-01,2020,1,1,4,61,41.0
1,02/01/2020,330,202,2020-01-02,2020,1,2,5,65,98.0
2,03/01/2020,737,360,2020-01-03,2020,1,3,6,174,233.0
3,04/01/2020,896,302,2020-01-04,2020,1,4,7,152,109.0
4,05/01/2020,552,342,2020-01-05,2020,1,5,1,145,170.0


Warning: We don't actually know if sales normal choc is in addition to chocolate_normal or if it is a part of it. Looking at the data, both options are possible. We assume for now that they are separate and to get the whole data we have to add the values together. In reality you would have to get that information from the original source before doing this addition.

In [15]:
df_combo2["chocolate_normal_combined"] = df_combo2["chocolate_normal"] + df_combo2["sales normal choc"]

In [16]:
df_combo2.head()

,date,chocolate_normal,chocolate_fancy,datetime,year,month,day,weekday,chocolate_frozen,sales normal choc,chocolate_normal_combined
0,01/01/2020,213,167,2020-01-01,2020,1,1,4,61,41.0,254.0
1,02/01/2020,330,202,2020-01-02,2020,1,2,5,65,98.0,428.0
2,03/01/2020,737,360,2020-01-03,2020,1,3,6,174,233.0,970.0
3,04/01/2020,896,302,2020-01-04,2020,1,4,7,152,109.0,1005.0
4,05/01/2020,552,342,2020-01-05,2020,1,5,1,145,170.0,722.0


The dataframe is a bit chaotic and contains columns we no longer need, so lets create a new one with just the information we need

In [17]:
final_solution = df_combo2[['datetime', 'year', 'month', 'day', 'weekday','chocolate_normal_combined','chocolate_fancy','chocolate_frozen']]
final_solution.head()

,datetime,year,month,day,weekday,chocolate_normal_combined,chocolate_fancy,chocolate_frozen
0,2020-01-01,2020,1,1,4,254.0,167,61
1,2020-01-02,2020,1,2,5,428.0,202,65
2,2020-01-03,2020,1,3,6,970.0,360,174
3,2020-01-04,2020,1,4,7,1005.0,302,152
4,2020-01-05,2020,1,5,1,722.0,342,145


In [18]:
final_solution.set_index('datetime', inplace=True)

In [19]:
final_solution.rename(columns={"chocolate_normal_combined": "chocolate_normal"})

,year,month,day,weekday,chocolate_normal,chocolate_fancy,chocolate_frozen
datetime,,,,,,,
2020-01-01,2020,1,1,4,254.0,167,61
2020-01-02,2020,1,2,5,428.0,202,65
2020-01-03,2020,1,3,6,970.0,360,174
2020-01-04,2020,1,4,7,1005.0,302,152
2020-01-05,2020,1,5,1,722.0,342,145
...,...,...,...,...,...,...,...
2020-12-27,2020,12,27,1,744.0,441,365
2020-12-28,2020,12,28,2,685.0,320,194
2020-12-29,2020,12,29,3,561.0,415,211


In [20]:
final_solution.to_csv('chocolate_combined_solution.csv')